In [ ]:
import os
import subprocess
import psutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import librosa
import mlflow
import mlflow.pytorch
from pynvml import (
    nvmlInit, nvmlDeviceGetHandleByIndex,
    nvmlDeviceGetUtilizationRates, nvmlDeviceGetMemoryInfo,
    nvmlDeviceGetTemperature, NVML_TEMPERATURE_GPU
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

mlflow.set_experiment("Panns_CNN14_FineTune")
try:
    mlflow.end_run()
except:
    pass
mlflow.start_run(log_system_metrics=True)

# log GPU info
gpu_info = next(
    (subprocess.run(cmd, capture_output=True, text=True).stdout
        for cmd in ["nvidia-smi", "rocm-smi"]
        if subprocess.run(f"command -v {cmd}", shell=True,
                          capture_output=True).returncode == 0),
    "No GPU found."
)
mlflow.log_text(gpu_info, "gpu-info.txt")

nvmlInit()
gpu_handle = nvmlDeviceGetHandleByIndex(0)

def log_system_metrics_mlflow(step=None):
    mlflow.log_metric("system.cpu.utilization", psutil.cpu_percent(), step=step)
    mem = psutil.virtual_memory()
    mlflow.log_metric("system.memory.used", mem.used, step=step)
    mlflow.log_metric("system.memory.percent", mem.percent, step=step)
    gpu_util = nvmlDeviceGetUtilizationRates(gpu_handle).gpu
    mlflow.log_metric("system.gpu.0.utilization", gpu_util, step=step)
    gpu_mem = nvmlDeviceGetMemoryInfo(gpu_handle)
    mlflow.log_metric("system.gpu.0.memory.used", gpu_mem.used, step=step)
    mlflow.log_metric("system.gpu.0.memory.percent",
                      (gpu_mem.used / gpu_mem.total) * 100, step=step)
    gpu_temp = nvmlDeviceGetTemperature(gpu_handle, NVML_TEMPERATURE_GPU)
    mlflow.log_metric("system.gpu.0.temperature", gpu_temp, step=step)

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, manifest_csv: str, metadata_csv: str,
                 sr: int = 32000, duration: float = 10.0):
        """
        manifest_csv: path to manifest_train.csv or manifest_test.csv
        metadata_csv: path to full train.csv (for secondary_labels)
        sr: sample rate to load audio
        duration: length (s) to pad/truncate each file
        """
        m_df = pd.read_csv(manifest_csv)
        meta = pd.read_csv(metadata_csv, usecols=["filename","secondary_labels"])
        meta["recording_id"] = meta.filename.str.replace(r"\.ogg$", "", regex=True)
        meta["sec_list"]     = meta.secondary_labels.fillna("").str.split()
        sec_map = dict(zip(meta.recording_id, meta.sec_list))

        self.rows = []
        all_labels = set()
        self.sr = sr
        self.num_samples = int(sr * duration)

        for _, row in tqdm(m_df.iterrows(),
                            total=len(m_df),
                            desc=f"Building AudioDataset ({os.path.basename(manifest_csv)})"):
            rid    = row.chunk_id.split("_chk")[0]
            labs   = [row.primary_label] + sec_map.get(rid, [])
            all_labels.update(labs)
            self.rows.append({
                "audio_path": row.audio_path,
                "labels": labs
            })

        self.classes   = sorted(all_labels)
        self.label2idx = {l:i for i,l in enumerate(self.classes)}
        self.num_classes = len(self.classes)

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
        r = self.rows[idx]
        # load waveform
        wav, _ = librosa.load(r["audio_path"], sr=self.sr)
        # pad or truncate
        if len(wav) < self.num_samples:
            pad = self.num_samples - len(wav)
            wav = np.pad(wav, (0,pad), mode="constant")
        else:
            wav = wav[:self.num_samples]
        x = torch.from_numpy(wav).unsqueeze(0).float()  # (1, num_samples)

        # multi‑hot target
        y = torch.zeros(self.num_classes, dtype=torch.float32)
        for l in r["labels"]:
            y[self.label2idx[l]] = 1.0

        return x, y

In [ ]:
panns = torch.hub.load(
    "qiuqiangkong/audioset_tagging_cnn", "Cnn14",
    pretrained=True, 
    classes_num=None       # we'll override the head below
)

# replace audio-tagging head for our num_classes
# original PANNs has attr 'fc2' or 'fc_audioset'; here we use fc2
in_features = panns.fc2.in_features if hasattr(panns, "fc2") else panns.fc_audioset.in_features
new_head = nn.Linear(in_features, None)  # placeholder
# but since we want dynamic assignment, we'll assign after dataset creation

In [ ]:
TRAIN_MANIFEST = "/home/jovyan/Features/manifest_train.csv"
TEST_MANIFEST  = "/home/jovyan/Features/manifest_test.csv"
TRAIN_CSV      = "/home/jovyan/Data/birdclef-2025/train.csv"

BATCH_SIZE = 8     # smaller batch for long audio
LR         = 1e-4
EPOCHS     = 20

train_ds = AudioDataset(TRAIN_MANIFEST, TRAIN_CSV)
test_ds  = AudioDataset(TEST_MANIFEST,  TRAIN_CSV)

# now override PANNs head with correct num_classes
num_classes = train_ds.num_classes
if hasattr(panns, "fc2"):
    panns.fc2 = nn.Linear(panns.fc2.in_features, num_classes)
else:
    panns.fc_audioset = nn.Linear(panns.fc_audioset.in_features, num_classes)
model = panns.to(device)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4, pin_memory=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

mlflow.log_params({
    "model":        "PANNs_Cnn14_finetune",
    "input":        "waveform",
    "sr":           train_ds.sr,
    "duration_s":   train_ds.num_samples / train_ds.sr,
    "num_classes":  num_classes,
    "batch_size":   BATCH_SIZE,
    "lr":           LR,
    "epochs":       EPOCHS
})

In [ ]:
best_test_acc = 0.0
best_ckpt     = None

for epoch in range(1, EPOCHS+1):
    # — Train —
    model.train()
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Train", unit="batch")
    run_loss, correct, total = 0.0, 0, 0

    for xb, yb in train_bar:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)         # [B, num_classes]
        loss   = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        run_loss += loss.item() * xb.size(0)
        preds = (torch.sigmoid(logits) > 0.5).float()
        correct += (preds == yb).all(dim=1).sum().item()
        total   += xb.size(0)

        train_bar.set_postfix({
            "loss": f"{run_loss/total:.4f}",
            "acc":  f"{correct/total:.4f}"
        })

    train_loss = run_loss/total
    train_acc  = correct/total

    # — Test —
    model.eval()
    test_bar = tqdm(test_loader, desc=f"Epoch {epoch} Test ", unit="batch")
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for xb, yb in test_bar:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss   = criterion(logits, yb)

            test_loss += loss.item() * xb.size(0)
            preds = (torch.sigmoid(logits) > 0.5).float()
            correct += (preds == yb).all(dim=1).sum().item()
            total   += xb.size(0)

            test_bar.set_postfix({
                "loss": f"{test_loss/total:.4f}",
                "acc":  f"{correct/total:.4f}"
            })

    test_loss /= total
    test_acc   = correct/total

    # checkpoint
    ckpt = f"panns_cnn14_epoch_{epoch}.pt"
    torch.save({
        "epoch":       epoch,
        "model_state": model.state_dict(),
        "optim_state": optimizer.state_dict(),
        "train_loss":  train_loss,
        "test_loss":   test_loss
    }, ckpt)

    mlflow.log_metrics({
        "train_loss":     train_loss,
        "train_accuracy": train_acc,
        "test_loss":      test_loss,
        "test_accuracy":  test_acc
    }, step=epoch)
    log_system_metrics_mlflow(step=epoch)
    mlflow.log_artifact(ckpt, artifact_path="checkpoints")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_ckpt     = ckpt

    print(f"→ Epoch {epoch}/{EPOCHS}  "
          f"Train loss={train_loss:.4f}, acc={train_acc:.4f} │ "
          f"Test loss={test_loss:.4f}, acc={test_acc:.4f}")

In [ ]:
mlflow.log_metric("best_test_accuracy", best_test_acc)
mlflow.log_artifact(best_ckpt, artifact_path="model")
mlflow.pytorch.log_model(model, "panns_cnn14_model")
mlflow.end_run()